In [1]:
cd ..

/Users/deepakduggirala/Documents/autonomous-robotics/translating-images-into-maps


In [22]:
import os
import io

import pickle

import numpy as np

import lmdb
from PIL import Image
import torch

from pathlib import Path

from nuscenes.nuscenes import NuScenes
from torchvision.transforms.functional import to_tensor, to_pil_image

In [3]:
from src.data.dataloader import nuScenesMaps, read_split

In [4]:
data_root = Path.resolve(Path('/Users/deepakduggirala/Documents/autonomous-robotics/v1.0-trainval_meta/'))
nusc = NuScenes(version='v1.0-trainval', dataroot=data_root, verbose=False)
tokens = read_split(
            os.path.join(data_root, "splits", "{}.txt".format('evaluation'))
        )

In [5]:
def image_to_bytes(im):
    byteImgIO = io.BytesIO()
    im.save(byteImgIO, "PNG")
    return byteImgIO.getvalue()

def image_from_bytes(img_bytes):
    return Image.open(io.BytesIO(img_bytes)).convert(mode='RGB')

def img_equal(im1, im2):
    return torch.all(to_tensor(im1) == to_tensor(im2))

In [6]:
# needs nusc, data_root
def get_cam_front_path(sample_token):
    sample = nusc.get('sample', sample_token)
    cam_front_token = sample['data']['CAM_FRONT']
    cam_front_data = nusc.get('sample_data', cam_front_token)
    cam_front_path = (data_root / Path(cam_front_data['filename']))
    return cam_front_path


def get_key_value(sample_token):
    cam_front_path = get_cam_front_path(sample_token)
    
    im = Image.open(str(cam_front_path))
    im_bytes = image_to_bytes(im)
    
    key_bytes = bytes(cam_front_path.stem, 'utf-8')
    
    return key_bytes, im_bytes

In [7]:
images_db_path = os.path.join(
            data_root, "lmdb",
            "samples", "CAM_FRONT"
        )

In [33]:
%%time
total_bytes = 0
with lmdb.open(path=images_db_path, map_size = int(1 * 1024 * 1024 * 1024)) as images_db:
    with images_db.begin(write=True) as txn:
        for i,sample_token in enumerate(tokens):
            key, value = get_key_value(sample_token)
            total_bytes += len(key) + len(value)
            txn.put(key, value)
            if (i+1)%10==0:
                print(f'{i+1} images put into lmdb. {total_bytes/(1024*1024):.2f} MB written.')

10 images put into lmdb. 8.46 MB written.
20 images put into lmdb. 17.37 MB written.
30 images put into lmdb. 27.51 MB written.
40 images put into lmdb. 37.07 MB written.
50 images put into lmdb. 49.32 MB written.
60 images put into lmdb. 59.09 MB written.
CPU times: user 17.7 s, sys: 143 ms, total: 17.8 s
Wall time: 18.3 s


In [ ]:
key = get_cam_front_path(tokens[50]).stem.encode('utf-8')
images_db_path='/Users/deepakduggirala/Documents/autonomous-robotics/v1.0-mini/lmdb/samples/CAM_FRONT'
with lmdb.open(path=str(images_db_path), 
               readonly=True, 
               readahead=False,
               max_spare_txns=128,) as images_db:
    with images_db.begin() as txn:
        im_read = txn.get(key)
        
image_from_bytes(im_read)

In [25]:
def get_cam_front_data(sample_token):
    sample = nusc.get('sample', sample_token)
    cam_front_token = sample['data']['CAM_FRONT']
    cam_front_data = nusc.get('sample_data', cam_front_token)
    calib = nusc.get("calibrated_sensor", cam_front_data["calibrated_sensor_token"])["camera_intrinsic"]
    
    return {
        'filename': cam_front_data['filename'], 
        'calib': calib
    }
eval_data = {token:get_cam_front_data(token) for token in tokens}

In [28]:
with open('eval_data.pickle', 'wb') as f:
    pickle.dump(eval_data, f)

In [32]:
filenames = [data['filename'] for data in eval_data.values()]
print(' '.join([f"'{filename}'" for filename in filenames]))

'samples/CAM_FRONT/n008-2018-09-18-15-12-01-0400__CAM_FRONT__1537298218412404.jpg' 'samples/CAM_FRONT/n015-2018-07-24-11-03-52+0800__CAM_FRONT__1532401508412460.jpg' 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657117612404.jpg' 'samples/CAM_FRONT/n008-2018-09-18-15-12-01-0400__CAM_FRONT__1537298355612404.jpg' 'samples/CAM_FRONT/n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731138162404.jpg' 'samples/CAM_FRONT/n008-2018-09-18-13-10-39-0400__CAM_FRONT__1537290827412404.jpg' 'samples/CAM_FRONT/n008-2018-09-18-13-10-39-0400__CAM_FRONT__1537290892912404.jpg' 'samples/CAM_FRONT/n008-2018-09-18-15-12-01-0400__CAM_FRONT__1537298128162404.jpg' 'samples/CAM_FRONT/n015-2018-07-18-11-50-34+0800__CAM_FRONT__1531885919612469.jpg' 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657278662404.jpg' 'samples/CAM_FRONT/n008-2018-09-18-13-10-39-0400__CAM_FRONT__1537290844162404.jpg' 'samples/CAM_FRONT/n008-2018-09-18-14-43-59-0400__CAM_FRONT__1537296540162404.jpg' 'sam